### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import h5py

import preprocessing_utils as utils

# CaloChallenge Preprocessing


The data preprocessing consists of :
1. Same impact energie and # points:

    1.1. Conversion in GeV  \
    1.2. impact energy range [ 1, 1000 ] GeV  \

2. Smearing

3. Reordering by increasing # points 

4. Removing sampling fraction rescaling:
 multiply point energies by 0.033

The goal is to make as much as possible 
In the cell below if `train` is activated we

In [ ]:
# Loading Data
# n_datasets = [1, 2, 3] -> for the training
# n_datasets = [4] -> for the testing

train = True

if train:
    n_datasets = [1]
    print("Loading the training data...\n")
else:
    n_datasets = [4]
    print("Loading the testing data...\n")


data = utils.loading_files(path = "/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/original", 
                           n_datasets = n_datasets)

## PreProcessing

### 1. Filter

#### 1.1 conversion in GeV

In [ ]:
# Normalizing the data: conversion in GeV

data = utils.normalize_incident(data)

#### 1.2-1.3 Impact energies in the range chosen impact energy range GeV 

In [ ]:
filtered_data = utils.filter_events(data,  energy_range=(500, 800))
for i in range(len(filtered_data)):
    print()
    print(filtered_data[i]['showers'].shape)

if you want to select only some indices

In [ ]:
if train is False:
    # Generate random indices

    # 1. Prendo il dizionario che sta dentro la lista
    data = filtered_data[0]

    # 2. Genero 10.000 indici unici basati sul numero di righe di 'showers'
    indices = np.random.choice(data['showers'].shape[0], size=10000, replace=False)

    # 3. Campiono entrambe le matrici usando gli stessi indici
    data['showers']  = data['showers'][indices]
    data['incident'] = data['incident'][indices]
    print(data['showers'].shape)

In [ ]:
# Converting the data in point cloud
point_cloud = utils.to_point_cloud(filtered_data)
# del data, n_datasets
utils.free_memory()

In [ ]:
utils.plt_scatter(point_cloud['showers'][-1])
utils.plt_scatter_2(point_cloud['showers'][-1])
# utils.plt_scatter_2(utils.to_cylindrical(point_cloud['showers'][1]), cylindrical=True, title="Cylindrical coordinates")

### 2. Smearing

In [ ]:
plt.rcParams.update({
        # Use a serif font that's likely available
        'font.family': 'serif',
        'font.serif': ['DejaVu Serif', 'Liberation Serif', 'Computer Modern Roman', 'Bitstream Vera Serif'],
        'font.size': 12,
        'axes.labelsize': 14,
        'axes.titlesize': 16,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'legend.fontsize': 12,
        'figure.dpi': 300,
        'savefig.dpi': 600,  # Higher DPI for publication quality
        'savefig.format': 'pdf',  # PDF format is often preferred for publications
        'savefig.bbox': 'tight',
        'savefig.pad_inches': 0.1,
        'axes.linewidth': 0.8,  # Slightly thinner axes lines
        'lines.linewidth': 1.5,  # Slightly thicker plot lines
        'lines.markersize': 4,  # Slightly smaller markers
        'axes.grid': True,
        'grid.alpha': 0.3
    })

In [ ]:
for i in range(len(point_cloud['showers'])):
    if 500 <= point_cloud['incident'][i] <= 501:
        print(i)


In [ ]:
print('Shape of the showers:', point_cloud['showers'].shape)
for i in range(3):
    print('events in {}:'.format(i), point_cloud['showers'][:, i].min(), point_cloud['showers'][:, i].max())
    
utils.free_memory()

if train: 
    print('=== incident energy:', point_cloud['incident'][209])

    utils.plt_scatter_2(point_cloud['showers'][209], title='Not smeared')

    cylindrical_smear = utils.data_smearing(point_cloud['showers'][:])
    # smeared = utils.data_smearing(point_cloud['showers'][:209209], noise=209.35)
    utils.plt_scatter_2(cylindrical_smear[209], title='w/ Cylindrical smearing')
    
    
    for i in range(3):
        print('events in {} after the smearing:'.format(i), cylindrical_smear[:, i].min(), cylindrical_smear[:, i].max())
else:
    utils.plt_scatter_2(point_cloud['showers'][10], title='Evaluation data - Not smeared')
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch

# Your existing matplotlib configuration
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['DejaVu Serif', 'Liberation Serif', 'Computer Modern Roman', 'Bitstream Vera Serif'],
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'savefig.dpi': 600,
    'savefig.format': 'pdf',
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
    'axes.linewidth': 0.8,
    'lines.linewidth': 1.5,
    'lines.markersize': 4,
    'axes.grid': True,
    'grid.alpha': 0.3
})

def plt_smearing_comparison(original_shower, smeared_shower):
    """Plot before and after cylindrical smearing in XZ plane only"""
    
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    # Original data (left plot) - XZ plane
    x_orig = original_shower[0, :]
    z_orig = original_shower[2, :]
    energy_orig = original_shower[3, :]  # Energy values for coloring
    
    # Smaller point sizes
    sizes_orig = energy_orig * 0.2   # Much smaller scaling
    
    # Get the energy range for proper colorbar scaling
    energy_min = min(energy_orig.min(), smeared_shower[3, :].min())
    energy_max = max(energy_orig.max(), smeared_shower[3, :].max())
    
    print(f"Energy range: {energy_min:.2f} to {energy_max:.2f}")
    
    scatter1 = axs[0].scatter(x_orig, z_orig, s=sizes_orig, c=energy_orig, 
                             alpha=0.8, cmap='plasma', vmin=energy_min, vmax=energy_max)
    axs[0].set_ylim(-18, 18)
    axs[0].set_xlim(-18, 18)
    axs[0].set_title('Before Smearing')
    axs[0].set_aspect('equal')  # Balanced circles
    
    # Remove cartesian axes
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['bottom'].set_visible(False)
    axs[0].spines['left'].set_visible(False)
    
    # Add concentric circles and radial lines
    num_segments_x = 18
    num_segments_z = 50
    for radius in range(1, num_segments_x):
        circle = plt.Circle((0, 0), radius, color='black', alpha=0.5, 
                          fill=False, linewidth=0.8)
        axs[0].add_artist(circle)
    
    # Last circle with thicker line
    circle = plt.Circle((0, 0), num_segments_x, color='black', alpha=0.5, 
                       fill=False, linewidth=2.0)
    axs[0].add_artist(circle)
    
    # Add radial lines (central lines made thinner) - ALL BLACK
    theta = np.linspace(0, 2*np.pi, num_segments_z + 1)
    for i in range(num_segments_z):
        axs[0].plot([0, 18*np.cos(theta[i])], [0, 18*np.sin(theta[i])], 
                   color='black', alpha=0.5, linewidth=0.4)
    
    # Smeared data (right plot) - XZ plane
    x_smear = smeared_shower[0, :]
    z_smear = smeared_shower[2, :]
    energy_smear = smeared_shower[3, :]  # Energy values for coloring
    
    # Smaller point sizes
    sizes_smear = energy_smear * 0.3 + 0.2  # Much smaller scaling
    
    scatter2 = axs[1].scatter(x_smear, z_smear, s=sizes_smear, c=energy_smear, 
                             alpha=1, cmap='plasma', vmin=energy_min, vmax=energy_max)
    axs[1].set_ylim(-18, 18)
    axs[1].set_xlim(-18, 18)
    axs[1].set_title('After Smearing')
    axs[1].set_aspect('equal')  # Balanced circles
    
    # Remove cartesian axes
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['bottom'].set_visible(False)
    axs[1].spines['left'].set_visible(False)
    
    # Add concentric circles and radial lines for smeared
    for radius in range(1, num_segments_x):
        circle = plt.Circle((0, 0), radius, color='black', alpha=0.5, 
                          fill=False, linewidth=0.8)
        axs[1].add_artist(circle)
    
    # Last circle with thicker line
    circle = plt.Circle((0, 0), num_segments_x, color='black', alpha=0.5, 
                       fill=False, linewidth=2.0)
    axs[1].add_artist(circle)
    
    # Add radial lines (central lines made thinner) - ALL BLACK
    for i in range(num_segments_z):
        axs[1].plot([0, 18*np.cos(theta[i])], [0, 18*np.sin(theta[i])], 
                   color='black', alpha=0.5, linewidth=0.4)
    
    # Turn off grid for both
    for i in range(2):
        axs[i].grid(False)
    
    # Add colorbar positioned to the right of the second plot only
    cbar = plt.colorbar(scatter2, ax=axs[1], orientation='vertical', 
                       fraction=0.046, pad=0.1, shrink=1.0)
    cbar.set_label('Energy [MeV]', rotation=270, labelpad=15)
    
    # Explicitly set colorbar limits to match actual data range
    cbar.mappable.set_clim(vmin=energy_min, vmax=energy_max)
    
    # Add arrow between plots properly centered
    arrow = FancyArrowPatch((0.44, 0.5), (0.56, 0.5),
                           connectionstyle="arc3,rad=0", arrowstyle='->',
                           mutation_scale=25, color='black',
                           transform=fig.transFigure)
    fig.patches.append(arrow)
    
    fig.subplots_adjust(wspace=0.3)
    plt.savefig('./results/for_paper/smearing_comparison.pdf', bbox_inches='tight', pad_inches=0.1)
    plt.show()

# Your existing code with the new function
print('Shape of the showers:', point_cloud['showers'].shape)
for i in range(3):
    print('events in {}:'.format(i), point_cloud['showers'][:, i].min(), point_cloud['showers'][:, i].max())
    
utils.free_memory()

if train: 
    print('=== incident energy:', point_cloud['incident'][209])
    
    cylindrical_smear = utils.data_smearing(point_cloud['showers'][:])
    
    # Create the comparison plot
    plt_smearing_comparison(point_cloud['showers'][209], cylindrical_smear[209])
    
    for i in range(3):
        print('events in {} after the smearing:'.format(i), cylindrical_smear[:, i].min(), cylindrical_smear[:, i].max())
else:
    utils.plt_scatter_2(point_cloud['showers'][10], title='Evaluation data - Not smeared')

### 3. Reordering by npoints

In [ ]:
# if the training dataset is used, we consider the cylindrical smearing 
sorted_data, max_npoints = utils.sort_and_process(cylindrical_smear if train else point_cloud['showers'],
                                                    point_cloud['incident'])

In [ ]:

# Extract ordered data
sorted_dataz = [np.count_nonzero(point_cloud['showers'][i, -1, :]) for i in range(len(point_cloud['showers']))]
indices = list(range(len(sorted_dataz)))

# Create the figure and axes for two plots
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Plot 1
ax[0].scatter(x=indices, y=sorted_dataz, marker='o', color='r', s=1)
ax[0].set_xlabel('Indices')
ax[0].set_ylabel('# points')
ax[0].set_title('Ordered showers by # points')
ax[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend with maximum and average values for the first plot
min_value_1 = np.min(sorted_dataz)
max_value_1 = np.max(sorted_dataz)
avg_value_1 = np.mean(sorted_dataz)
ax[0].legend([f'Min: {min_value_1} \nMax: {max_value_1} \nAvg: {avg_value_1:.2f}']).set_title('# points')

# Example data for the second plot
# We need to define another set of data. In this example, I will use existing data.
sorted_en = [point_cloud['incident'][i] for i in range(len(point_cloud['incident']))]

# Plot 2
ax[1].scatter(indices, sorted_en, marker='o', color='r', s=1)
ax[1].set_xlabel('Indices')
ax[1].set_ylabel('Incident Energies [GeV]')
ax[1].set_title('Ordered incident energies by # points')
ax[1].grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend with maximum and average values for the second plot
min_value_2 = np.min(sorted_en)
max_value_2 = np.max(sorted_en)
avg_value_2 = np.mean(sorted_en)
ax[1].legend([f'Min: {min_value_2:.2f} \nMax: {max_value_2:.2f} \nAvg: {avg_value_2:.2f}']).set_title('Incidents E')

# Show both plots side by side
plt.tight_layout()
plt.show()

In [ ]:

# Extract ordered data
sorted_data_2 = [np.count_nonzero(sorted_data['showers'][i, -1, :]) for i in range(len(sorted_data['showers']))]
indices = list(range(len(sorted_data_2)))

# Create the figure and axes for two plots
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Plot 1
ax[0].scatter(x=indices, y=sorted_data_2, marker='o', color='r', s=1)
ax[0].set_xlabel('Indices')
ax[0].set_ylabel('# points')
ax[0].set_title('Ordered showers by # points')
ax[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend with maximum and average values for the first plot
min_value_1 = np.min(sorted_data_2)
max_value_1 = np.max(sorted_data_2)
avg_value_1 = np.mean(sorted_data_2)
ax[0].legend([f'Min: {min_value_1} \nMax: {max_value_1} \nAvg: {avg_value_1:.2f}']).set_title('# points')

# Example data for the second plot
# We need to define another set of data. In this example, I will use existing data.
sorted_en = [sorted_data['incident'][i] for i in range(len(sorted_data['incident']))]

# Plot 2
ax[1].scatter(indices, sorted_en, marker='o', color='r', s=1)
ax[1].set_xlabel('Indices')
ax[1].set_ylabel('Incident Energies [GeV]')
ax[1].set_title('Ordered incident energies by # points')
ax[1].grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend with maximum and average values for the second plot
min_value_2 = np.min(sorted_en)
max_value_2 = np.max(sorted_en)
avg_value_2 = np.mean(sorted_en)
ax[1].legend([f'Min: {min_value_2:.2f} \nMax: {max_value_2:.2f} \nAvg: {avg_value_2:.2f}']).set_title('Incidents E')

# Show both plots side by side
plt.tight_layout()
plt.show()

### 4. Rescaling energy

In [ ]:
# del sorted_simple_smearing, sorted_incident_energy, sorted_indices, sorted_npoints, npoints
utils.free_memory()

In [ ]:
preprocessed_data = utils.rescaling_e(sorted_data)
print(f"Shape of preprocessed data: {preprocessed_data['showers'].shape}")

In [ ]:
for i in range(3):
    print('events in {}:'.format(i), preprocessed_data['showers'][:, i].min(), preprocessed_data['showers'][:, i].max())

In [ ]:
utils.plt_scatter_2(preprocessed_data['showers'][-1], title='Preprocessed data')


## save validation file

In [ ]:
showers = preprocessed_data['showers'].copy()

hist_val = np.zeros((showers.shape[0], 45, 50, 18))

Xmin, Xmax = -18, 18
Ymin, Ymax = 0, 45
Zmin, Zmax = -18, 18

showers[:, 0, :] = (showers[:, 0, :] - Xmin) / (Xmax - Xmin) * 2 - 1
showers[:, 1, :] = (showers[:, 1, :] - Ymin) / (Ymax - Ymin) * 2 - 1
showers[:, 2, :] = (showers[:, 2, :] - Zmin) / (Zmax - Zmin) * 2 - 1


val_events_reshaped = np.zeros((showers.shape[0], 45 * 50 * 18))
for i in tqdm(range(showers.shape[0]), desc='reshaping into cylindrical'):
    hist = utils.cylindrical_histogram(showers[i])
    hist_reshaped = hist.reshape(45 * 50 * 18)
    val_events_reshaped[i] = hist_reshaped

for i in range(3):
    print('events in {}:'.format(i), showers[:, i].min(), showers[:, i].max())
       

In [ ]:
def plt_visible_e(dataset, log_scale=True, title=''):
    """
    Plots a histogram of the incident energies in the dataset.
    """
    plt.figure(figsize=(7, 4))
    # TODO: add 
    # visible_energy = events[:, 3, :][events[:, 3, :] > 0]

    # Plot the histogram
    plt.hist(dataset, bins=np.logspace(np.log(1e-7), np.log(dataset.max()), 200, base=np.e), alpha=0.75, color='blue', edgecolor='black')
    
    # Plot the mean line
    mean_energy = dataset.mean()
    plt.axvline(mean_energy, color='r', linestyle='dashed', linewidth=1, label=f'Mean energy: {mean_energy:.2f} GeV')
    
    # Optionally set a logarithmic scale for the y-axis
    if log_scale:
        plt.yscale('log')
        plt.xscale('log')
    
    # Add labels and title
    plt.xlabel('Visible  Energy (MeV)')
    plt.ylabel('Number of Events')
    plt.title('Visible Energy' + title, fontsize=26)
    plt.legend()
    plt.grid(True, which="both", ls="--", linewidth=0.5)

    plt.xlim(1e-4, dataset.max())
    
    plt.show()

In [ ]:
print(f"Shape of the reshaped data: {val_events_reshaped.shape}")
visible_points = val_events_reshaped[:, 0:len(val_events_reshaped[0])][val_events_reshaped[:, 0:len(val_events_reshaped[0])] > 0]
plt_visible_e(visible_points, title='Visible energy in the events')
plt_visible_e(visible_points/0.033, title='Visible energy reshaped')

In [ ]:
val_data_dict = {
        'incident_energies': preprocessed_data['incident'],
        # 'showers': val_events_reshaped,
        'showers': showers,
    }
print(f"Shape of preprocessed data: {val_data_dict['showers'].shape}")
# utils.plt_scatter_2(val_data_dict['showers'][1], title='Evaluation data - Not smeared')


In [ ]:
tot_samples = len(showers)
num_samples = 10000 # choose the number of samples in the training dataset here

def get_percentage(total, part):
        return part / total

data_percentage = get_percentage(tot_samples, num_samples)
if num_samples != tot_samples : 
    
    print(f'the data percentage is: \n {(data_percentage*100):.4f} % of total showers {tot_samples}')
else:
    print('the total training dataset is considered')

if data_percentage < 1. and data_percentage > 0.:
    size = int ( tot_samples * data_percentage )
    idx = np.random.choice(tot_samples, size=size, replace=False)
    idx = np.sort(idx).astype(int)
    print(f'{(data_percentage*100):.2f} % of the dataset is considered!')
    print(idx)

In [ ]:
val_data_dict = {
        'incident_energies': preprocessed_data['incident'][idx],
        # 'showers': val_events_reshaped[idx],
        'showers': showers[idx],

    }
print(f"Shape of preprocessed data: {val_data_dict['showers'].shape}")
# utils.plt_scatter_2(val_data_dict['showers'][1], title='Evaluation data - Not smeared')


In [ ]:
plt.hist(val_data_dict['incident_energies'], bins=100, log=True, color='blue', alpha=0.7)

In [ ]:
# Save in an external file

val_file='10k_val_dset4_prep_1-1000GeV.hdf5'
folder = '1-1000GeV/evaluation/'
path = "/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/"
 
path + folder + val_file
utils.save_hdf5(val_data_dict, file_path= path + folder + val_file)

In [ ]:
eval_point_cloud = utils.to_point_cloud([val_data_dict])
# utils.plt_scatter_2(eval_point_cloud['showers'][1], title='Evaluation data')

In [ ]:
utils.plt_scatter_2(eval_point_cloud['showers'][-1], title='Evaluation data - Not smeared')

In [ ]:
utils.plt_scatter_2(eval_point_cloud['showers'][-1], title='Evaluation data - Not smeared')

In [ ]:
data = '/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/1-1000GeV/100k_train_dset1-2-3_prep_1-1000GeV.hdf5'
with h5py.File(data, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers = f['events'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_energy = f['energy'][:]
    print(f"Shape del dataset 'showers': {showers.shape}")   

In [ ]:
import h5py
data = '/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/1-1000GeV/evaluation/10k_val_dset4_prep_1-1000GeV.hdf5'
with h5py.File(data, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers = f['showers'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_energy = f['incident_energies'][:]
    print(f"Shape del dataset 'showers': {showers.shape}")   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
n_points = (showers[:][:, -1] > 0).sum(axis=1)
# print(f"Shape of the number of points: {n_points.shape}")
print(f"min and maximum number of points: {n_points.min()} and {n_points.max()}")
print(f'min max impact energy: {impact_energy.min()} and {impact_energy.max()}')
plt.hist(n_points, bins=100, log=True, color='blue', alpha=0.7)

### Comparing in the baseline value for Wasserstein and KL

In [ ]:
import h5py

# Specifica il percorso del file
file_path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/val_dset4-prep_0-500GeV.hdf5'

# Apri il file utilizzando h5py
with h5py.File(file_path, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers_dset4 = f['showers'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_dset4 = f['incident_energies'][:]
    print(f"Shape del dataset 'showers': {showers_dset4.shape}")        
showers_dset4 /=0.033
# showers_pc= utils.to_point_cloud([showers])

In [ ]:
import h5py

# Specifica il percorso del file
file_path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/val_dset3-prep_0-500GeV.hdf5'

# Apri il file utilizzando h5py
with h5py.File(file_path, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers_dset3 = f['showers'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_dset3 = f['incident_energies'][:]
    print(f"Shape del dataset 'showers': {showers_dset3.shape}") 
showers_dset3 /=0.033
       
# showers_pc= utils.to_point_cloud([showers])

In [ ]:
indices = np.random.choice(len(showers_dset3) , size=len(showers_dset4), replace=False)

showers_3_4_numpy = np.array([ showers_dset4, showers_dset3[indices]])
showers_3_4_numpy[0].shape, showers_3_4_numpy[1].shape

In [ ]:
import utils.plot_evaluate as plot
kl_divergences, wasserstein_dist = {}, {}
kl_divergences, wasserstein_dist = plot.plot_visible_energy( showers_3_4_numpy, kl_divergences=kl_divergences, wasserstein=wasserstein_dist,log_scale = True)
kl_divergences, wasserstein_dist

In [ ]:
kl_divergences, wasserstein_dist = plot.plot_nhits(showers_3_4_numpy, kl_divergences = kl_divergences, wasserstein=wasserstein_dist )
kl_divergences, wasserstein_dist

In [ ]:
kl_divergences, wasserstein_dist = plot.plot_radial_energy(showers_3_4_numpy, kl_divergences = kl_divergences, wasserstein=wasserstein_dist )
kl_divergences, wasserstein_dist

In [ ]:
wasserstein_dist

In [ ]:
df_wass = plot.plot_dataframe(wasserstein_dist,'Wasserstein distance')

In [ ]:
import matplotlib.pyplot as plt

def plt_scatter(shower, title='Scatter Plots of Shower Coordinates'):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle( title , fontsize=26)

    color = 'darkred'

    axes[0].scatter(shower[0, :], shower[1, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[0].set_xlabel('x', fontsize=22)
    axes[0].set_ylabel('y', fontsize=22)
    axes[0].set_title('x vs y', fontsize=24)

    axes[1].scatter(shower[0, :], shower[2, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[1].set_xlabel('x', fontsize=22)
    axes[1].set_ylabel('z', fontsize=22)
    axes[1].set_title('x vs z', fontsize=24)

    axes[2].scatter(shower[1, :], shower[2, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[2].set_xlabel('y', fontsize=22)
    axes[2].set_ylabel('z', fontsize=22)
    axes[2].set_title('y vs z', fontsize=24)

    [axes[i].grid(False) for i in range(3)]

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Remove plt.show() so it doesn't automatically display the plot
    return fig

In [ ]:
showers_pc = eval_point_cloud['showers']
impact_energy = eval_point_cloud['incident']

In [ ]:
import imageio
import matplotlib.pyplot as plt

n_images = 25

# Assume showers and impact_energy are defined
indices = np.linspace(0, showers_pc.shape[0] - 1, n_images, dtype=int)
frames = []

for idx in indices:
    n_points = np.count_nonzero(showers_pc[idx, -1, :])
    title = f"Shower {idx} - Energy {impact_energy[idx]} GeV - Number of points {n_points}"
    print(title)
    fig = plt_scatter(showers_pc[idx], title=f"Shower {idx} - Energy {impact_energy[idx]} GeV - Number of points {n_points}")
    
    fig.canvas.draw()  # Ensure the drawing is complete
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    frames.append(image)
    plt.close(fig)  # Close the figure to free up resources

output_file = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/gif/val_dset4-prep_0-500GeV.gif'
imageio.mimsave(output_file, frames, fps=0.5)

print(f"GIF saved as {output_file}")

## normalisation and saving training data

In [ ]:
showers = preprocessed_data['showers'].copy()

Xmin, Xmax = -18, 18
Ymin, Ymax = 0, 45
Zmin, Zmax = -18, 18

showers[:, 0, :] = (showers[:, 0, :] - Xmin) *2 / (Xmax - Xmin)  - 1
showers[:, 1, :] = (showers[:, 1, :] - Ymin) *2 / (Ymax - Ymin)  - 1
showers[:, 2, :] = (showers[:, 2, :] - Zmin) *2 / (Zmax - Zmin)  - 1

In [ ]:
train_data_dict = {
        'energy': preprocessed_data['incident'][1000:],
        'events': showers[1000:],
    }
print(f"Shape of preprocessed data: {train_data_dict['events'].shape}")

In [ ]:
train_file='100k_train_dset1-2-3_prep_1-500GeV.hdf5'
folder = '1-1000GeV/'
path = "/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/"

utils.save_hdf5(train_data_dict, file_path= path + folder + train_file)

In [ ]:
keys, showers, energy = utils.read_hdf5_file2(path + folder + train_file)

In [ ]:
path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/train_dset1-2_prep_10-90GeV.hdf5'
_, energy, events = utils.read_hdf5_file2(path )

#### making smaller dataset, choosing sizes

In [ ]:
train_file='100k_train_dset1-2-3_prep_1-500GeV.hdf5'
folder = '1-1000GeV/'
path = "/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/"


In [ ]:
import h5py

# Specifica il percorso del file
# file_path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/100_train_dset1-2_prep_0-500GeV.hdf5'
file_path = path + folder + train_file
# Apri il file utilizzando h5py
with h5py.File(file_path, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers = f['events'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_energy = f['energy'][:]
    print(f"Shape del dataset 'showers': {showers.shape}")        

In [ ]:
tot_samples = len(showers)
num_samples = 1000 # choose the number of samples in the training dataset here

def get_percentage(total, part):
        return part / total

data_percentage = get_percentage(tot_samples, num_samples)
if num_samples != tot_samples : 
    
    print(f'the data percentage is: \n {(data_percentage*100):.4f} % of total {tot_samples}')
else:
    print('the total training dataset is considered')

if data_percentage < 1. and data_percentage > 0.:
    size = int ( tot_samples * data_percentage )
    idx = np.random.choice(tot_samples, size=size, replace=False)
    idx = np.sort(idx).astype(int)
    print(f'{(data_percentage*100):.2f} % of the dataset is considered!')
    print(idx)
    
else:
    data_percentage = 1
    idx = np.arange(2874, len(showers)).astype(int)
    print('total daset is considered!')

In [ ]:
train_data_dict = {
        'energy': impact_energy[idx],
        'events': showers[idx],
    }
print(f"Shape of preprocessed data: {train_data_dict['events'].shape}")

In [ ]:
plt.hist(impact_energy, bins=100, log=True, histtype='step',label= 'total samples' )
plt.hist(train_data_dict['energy'], bins=100, log=True,  label=f'{(data_percentage*100):.1f} % samples' )
plt.legend()
plt.title('Impact Energies')
plt.show()

In [ ]:
n_point= np.count_nonzero(train_data_dict['events'][:, -1, :], axis=-1)
print('min and max number or points:', np.min(n_point), np.max(n_point))
print('min and max energy:', f"{np.min(train_data_dict['energy'][:]):.10g}", f"{np.max(train_data_dict['energy'][:]):.10g}")


In [ ]:
# Saving the files
train_file='train_dset1-2-3_prep_1-1000GeV.hdf5'
path = "/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/"

file_path = path + folder +  '1k_' +   train_file#'{:.0f}'.format(data_percentage*100) + 'k_' +   train_file
file_path
utils.save_hdf5(train_data_dict, file_path= file_path )

### Checking train dataset

In [ ]:
npoint = [np.count_nonzero(train_data_dict['events'][i, -1, :]) for i in range(train_data_dict['events'].shape[0])]
print(np.min(npoint), np.max(npoint))

# visible_shower = train_data_dict['events'][(train_data_dict['events'] != 0) ]

visible_energy = train_data_dict['events'][:, 3, :][train_data_dict['events'][:, 3, :] > 0]

plt.hist(visible_energy , bins=np.logspace(np.log(1e-7), np.log(visible_energy.max()), 200, base=np.e))
print(visible_energy.max())
plt.xscale('log')
plt.yscale('log')
# plt.ylim(ymax=1e8)
plt.xlim(xmin=1e-4)

plt.xlabel('Visible Energy [MeV]')
plt.ylabel('\# cell')

plt.legend(fontsize=20, frameon=True)

plt.show()

In [ ]:
import h5py

# Specifica il percorso del file
file_path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/train_dset1-2_prep_0-500GeV.hdf5'

# Apri il file utilizzando h5py
with h5py.File(file_path, 'r') as f:
    # Lista dei dataset disponibili
    print("Dataset disponibili nel file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers = f['events'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_energy = f['energy'][:]
    print(f"Shape del dataset 'showers': {showers.shape}")        

In [ ]:
import matplotlib.pyplot as plt

def plt_scatter(shower, title='Scatter Plots of Shower Coordinates'):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle( title , fontsize=26)

    color = 'darkred'

    axes[0].scatter(shower[0, :], shower[1, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[0].set_xlabel('x', fontsize=22)
    axes[0].set_ylabel('y', fontsize=22)
    axes[0].set_title('x vs y', fontsize=24)

    axes[1].scatter(shower[0, :], shower[2, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[1].set_xlabel('x', fontsize=22)
    axes[1].set_ylabel('z', fontsize=22)
    axes[1].set_title('x vs z', fontsize=24)

    axes[2].scatter(shower[1, :], shower[2, :], s=10, alpha=0.6, edgecolor='w', linewidth=0.5, color=color)
    axes[2].set_xlabel('y', fontsize=22)
    axes[2].set_ylabel('z', fontsize=22)
    axes[2].set_title('y vs z', fontsize=24)

    [axes[i].grid(False) for i in range(3)]

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Remove plt.show() so it doesn't automatically display the plot
    return fig

In [ ]:
import imageio
import matplotlib.pyplot as plt

n_images = 50

# Assume showers and impact_energy are defined
indices = np.linspace(0, showers.shape[0] - 1, n_images, dtype=int)
frames = []

for idx in indices:
    n_points = np.count_nonzero(showers[idx, -1, :])
    title = f"Shower {idx} - Energy {impact_energy[idx]} GeV - Number of points {n_points}"
    print(title)
    fig = plt_scatter(showers[idx], title=f"Shower {idx} - Energy {impact_energy[idx]} GeV - Number of points {n_points}")
    
    fig.canvas.draw()  # Ensure the drawing is complete
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    frames.append(image)
    plt.close(fig)  # Close the figure to free up resources

output_file = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/showers_animation.gif'
imageio.mimsave(output_file, frames, fps=1)

print(f"GIF saved as {output_file}")

In [ ]:
train, val = utils.split_and_save_hdf5(preprocessed_data, validation_size=10000, 
                                       save = False,
                                    train_file='train_prep_10-90GeV.hdf5',
                                    validation_file='val_prep_10-90GeV.hdf5')

## choosing a fraction of data

In [ ]:
import h5py

# Specifica il percorso del file
# file_path = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/train_dset1-2_prep_0-500GeV.hdf5'
file_path = '/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/10-90GeV/47k_dset1-2-3_prep_10-90GeV.hdf5'
# Apri il file utilizzando h5py
with h5py.File(file_path, 'r') as f:
    # Lista dei dataset disponibili
    print("Available keys in the file:")
    print(list(f.keys()))
    
    # Supponiamo che ci sia un dataset chiamato 'data'
    # Puoi leggere i tuoi dati nel modo seguente
    showers = f['events'][:]  # Questo scaricherà tutto il dataset in memoria
    impact_energy = f['energy'][:]
    print(f"Shape del dataset 'showers': {showers.shape}")        

In [ ]:
tot_samples = len(showers)
num_samples = 5_000 # choose the number of samples in the training dataset here

def get_percentage(total, part):
        return part / total

data_percentage = get_percentage(tot_samples, num_samples)
if num_samples != tot_samples : 
    
    print(f'the data percentage is: \n {(data_percentage*100):.4f} % of total showers {tot_samples}')
else:
    print('the total training dataset is considered')

if data_percentage < 1. and data_percentage > 0.:
    size = int ( tot_samples * data_percentage )
    idx = np.random.choice(tot_samples, size=size, replace=False)
    idx = np.sort(idx).astype(int)
    print(f'{(data_percentage*100):.2f} % of the dataset is considered!')
    print(idx)

In [ ]:
train_data_dict = {
        'energy': impact_energy[idx],
        'events': showers[idx],
    }
print(f"Shape of preprocessed data: {train_data_dict['events'].shape}")

In [ ]:
plt.hist(train_data_dict['energy'], bins=100, log=True, histtype='step',label= 'total samples' )
plt.show()

In [ ]:
# Saving the files
train_file='dset1-2-3_prep_10-90GeV.hdf5'
path = "/data/dust/user/valentel/maxwell.merged/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/10-90GeV/"

file_path = path + '5k' + '_' + train_file
utils.save_hdf5(train_data_dict, file_path= file_path )


## Sample choice for the training dataset

In [ ]:
# Assuming utils.read_hdf5_file2 is properly defined and provides correct data.
path_validation = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/val_dset4-prep_0-500GeV.hdf5'
_, _, incidents = utils.read_hdf5_file2(path_validation)
incidents_numpy = np.array(incidents).ravel()

sampling_sizes = [10000, 1000, 100]

# Create a figure with subplots
fig, axes = plt.subplots(nrows=1, ncols=len(sampling_sizes), figsize=(18, 6), sharex=True, sharey=True)
fig.suptitle('Analysis of Incident Energies with Different Sampling Sizes', fontsize=16)

# Iterate over each sampling size and subplot
for ax, size in zip(axes, sampling_sizes):
    x = incidents_numpy.copy()
    u = np.random.choice(x, size=size, replace=False)
    
    ax.hist(x, bins=100, log=True, histtype='step', label='Original')
    ax.hist(u, bins=100, log=True, histtype='step', label=f'{size} log rand choice')

    # Obtain histogram data to define bins
    h, b = np.histogram(x, bins=100)

    # Calculate bin indices and probabilities
    bin_indices = np.digitize(x, bins=b[:-1], right=True) - 1
    raw_probs = 1.0 / np.take(h, bin_indices, mode='clip')
    probs = raw_probs / raw_probs.sum()

    z = np.random.choice(x, size=size, replace=True, p=probs)  # Allow replacement since probabilities are used
    ax.hist(z, bins=100, log=True, histtype='step', label=f'{size} unif rand choice')

    ax.set_ylabel('Frequency (log scale)')
    ax.set_xlabel('Incident energies [GeV]')
    ax.legend(fontsize= 20)

# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # rect adjusts the space for the title
plt.show()

In [ ]:
# Assuming utils.read_hdf5_file2 is properly defined and provides correct data.
path_training = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/100_train_dset1-2_prep_0-500GeV.hdf5'
_, incidents, _ = utils.read_hdf5_file2(path_training)
incidents_numpy = np.array(incidents).ravel()

sampling_sizes = [10000, 1000 ]

# Create a figure with subplots
fig, axes = plt.subplots(nrows=1, ncols=len(sampling_sizes), figsize=(18, 6), sharex=True, sharey=True)
fig.suptitle('Analysis of Incident Energies with Different Sampling Sizes', fontsize=26)

# Iterate over each sampling size and subplot
for ax, size in zip(axes, sampling_sizes):
    x = incidents_numpy.copy()
    u = np.random.choice(x, size=size, replace=False)
    
    ax.hist(x, bins=100, log=True, histtype='step', label='Original')
    ax.hist(u, bins=100, log=True, histtype='step', label=f'{size} log rand choice')

    # Obtain histogram data to define bins
    h, b = np.histogram(x, bins=100)

    # Calculate bin indices and probabilities
    bin_indices = np.digitize(x, bins=b[:-1], right=True) - 1
    raw_probs = 1.0 / np.take(h, bin_indices, mode='clip')
    probs = raw_probs / raw_probs.sum()


    z = np.random.choice(x, size=size, replace=True, p=probs)  # Allow replacement since probabilities are used
    ax.hist(z, bins=100, log=True, histtype='step', label=f'{size} unif rand choice')

    ax.set_ylabel('Frequency (log scale)')
    ax.set_xlabel('Incident energies [GeV]')
    ax.legend(fontsize= 20)
utils.free_memory()
# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # rect adjusts the space for the title
plt.show()

In [ ]:
tot_samples = len(showers)
num_samples = 10000 # choose the number of samples in the training dataset here

def get_percentage(total, part):
        return part / total

data_percentage = get_percentage(tot_samples, num_samples)
if num_samples != tot_samples : 
    
    print(f'the data percentage is: \n {(data_percentage*100):.4f} % of total {tot_samples}')
else:
    print('the total training dataset is considered')

if data_percentage < 1. and data_percentage > 0.:
    size = int ( tot_samples * data_percentage )
    idx = np.random.choice(tot_samples, size=size, replace=False)
    idx = np.sort(idx).astype(int)
    print(f'{(data_percentage*100):.2f} % of the dataset is considered!')
    print(idx)

In [ ]:
# Assuming utils.read_hdf5_file2 is properly defined and provides correct data.
path_training = '/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/100_train_dset1-2_prep_0-500GeV.hdf5'
_, impact_energies, showers = utils.read_hdf5_file2(path_training)
impact_energies_numpy = np.array(impact_energies).ravel()

sampling_sizes = [1000, 10000]

# Create a figure with subplots
fig, axes = plt.subplots(nrows=1, ncols=len(sampling_sizes), figsize=(18, 6), sharex=True, sharey=True)
fig.suptitle('Analysis of Incident Energies with Different Sampling Sizes', fontsize=26)

# Iterate over each sampling size and subplot
for ax, size in zip(axes, sampling_sizes):
    x = impact_energies_numpy.copy()
    u_indices = np.random.choice(len(x), size=size, replace=False)
    u = x[u_indices]
    
    ax.hist(x, bins=100, log=True, histtype='step', label='Original')
    ax.hist(u, bins=100, log=True, histtype='step', label=f'{size} log rand choice')

    # Obtain histogram data to define bins
    h, b = np.histogram(x, bins=100)

    # Calculate bin indices and probabilities
    bin_indices = np.digitize(x, bins=b[:-1], right=True) - 1
    raw_probs = 1.0 / np.take(h, bin_indices, mode='clip')
    probs = raw_probs / raw_probs.sum()

    z_indices = np.random.choice(len(x), size=size, replace=True, p=probs)
    z_indices = np.sort(z_indices).astype(int)
    z = x[z_indices]
    ax.hist(z, bins=100, log=True, histtype='step', label=f'{size} unif rand choice')

    ax.set_ylabel('Frequency (log scale)')
    ax.set_xlabel('Incident energies [GeV]')
    ax.legend(fontsize=20)

    # Use the indices to select corresponding showers
    selected_showers_u = showers[u_indices]
    selected_showers_z = showers[z_indices]

utils.free_memory()
# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # rect adjusts the space for the title
plt.show()

In [ ]:
train_data_dict = {
        'energy': impact_energies[z_indices],
        'events': showers[z_indices],
    }
print(f"Shape of preprocessed data: {train_data_dict['events'].shape}")

In [ ]:
# Saving the files
train_file='uniform_train_dset1-2_prep_0-500GeV.hdf5'
path = "/gpfs/dust/maxwell/user/valentel/MyCaloTransfer/CaloTransfer/data/calo-challenge/preprocessing/reduced_datasets/"

file_path = path + '10k' + '_' + train_file
utils.save_hdf5(train_data_dict, file_path= file_path )